# Team Vaccinated: Milestone 1

## Part 1: Introduction

The goal of this practicum is to gather data from Facebook.

The following Python libraries are used:
 - Pandas: ...
 - Numpy: ...
 - BeautifulSoup: ...

## Part 2: Gather data

1. Open a page in Facebook.
2. Scroll down.
3. Ctrl+s to save page.

## Part 3: Run code

Run the code below to import the necessary libraries.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, NavigableString

In [2]:
#filename = 'Stop Mandatory Vaccination - Posts.html'
filename = 'HealthCare.gov - Posts.html'
number_of_posts = 3

In [3]:
f, full_soup = None, None

try:
    f = open(filename, encoding='utf-8')
    full_soup = BeautifulSoup(f)
except Exception as e:
    print(e)

In [4]:
#Parse a given post's soup
def parse_post(post_soup):
    data = {}
    
    #Get date published
    data['timestamp'] = post_soup.find('span', class_='fsm fwn fcg').find('span', class_='timestampContent').text
    
    #Get text and links on the post
    data['links'] = []
    data['text'] = ''
    for p in post_soup.find_all('p'):
        if p.a:
            a_tags = p.a.extract()
            data['links'].append(a_tags.text.strip())
        data['text'] += p.text.strip()
    
    #Find if it has an image or not
    img = post_soup.find('img', {'class': 'scaledImageFitWidth img'})
    data['img_src'] = img.attrs['src'] if img and 'src' in img.attrs else None
    data['img-label'] = img.attrs['aria-label'] if img and 'aria-label' in img.attrs else None
    
    #Get the link info if it exists
    article = post_soup.find('div', {'class': 'mbs'})
    data['article_name'] = article.text if article and 'text' in article else None
    host = post_soup.find('div', {'class': '_6lz _6mb _1t62 ellipsis'})
    data['article_host'] = host.text if host and 'text' in host else None
    subtitle = post_soup.find('div', class_='_6m7 _3bt9')
    data['article_subtitle'] = subtitle.text if subtitle else None
    
    #Find other profiles if it has linked to them
    data['linked_profiles'] = [page.text for page in post_soup.find_all('span', class_='fwb')
                               if page.text != 'Stop Mandatory Vaccination']
    
    return data

In [5]:
#Parse all posts given a soup
def parse_posts(soup, limit=0):
    for i, post_child in enumerate(soup.find_all('div', class_='userContent')):
        if limit != 0 and i >= limit:
            break
        yield parse_post(post_child.parent)

In [6]:
posts = list(parse_posts(full_soup, limit=number_of_posts))
print('Number of posts: {}'.format(len(posts)))

Number of posts: 3


In [9]:
import pprint
pp = pprint.PrettyPrinter(indent=4, width=110)
for p in posts:
    pp.pprint(p['text'])

(': Eating healthy is important at any age, and it’s easy too! Just a few small changes can make a BIG '
 'difference. Learn ways to find a healthy eating style that’s right for you:https://go.hc.gov/2NKUYWJ '
 '#NutritionMont')
('No better time than during winter to jump start your path to being healthy. And no better way than to take '
 'advantage of all the preventive services covered by the Marketplace:   #MondayMotivation')
('We’re wearing blue for , are you? Don’t forget, colorectal cancer screenings are covered under the '
 'Marketplace with no copayment or coinsurance.')
